# Introdução

## Três paradigmas para análise de portifólio com R

* *xts*
    - quantmod
    - PerformanceAnalytics
* *tidyverse* 
* *tidyquant*

In [38]:
library(timetk)

Warning message:
“package ‘timetk’ was built under R version 3.6.3”

ERROR: Error: package or namespace load failed for ‘timetk’:
 object 'required_pkgs' not found whilst loading namespace 'timetk'


In [21]:
# Pacotes importados
library(tidyverse)
library(lubridate)
library(readxl)
library(highcharter)
library(tidyquant)
library(timetk)
library(tibbletime)
library(quantmod)
library(PerformanceAnalytics)
library(scales)

Warning message:
“package ‘timetk’ was built under R version 3.6.3”

ERROR: Error: package or namespace load failed for ‘timetk’:
 object 'required_pkgs' not found whilst loading namespace 'timetk'


A função `getSymbols` pertence ao pacote `quantmod` e retorna um objeto com os seguintes dados referentes ao ativo representado por um dado *ticker*:

* preço de abertura
* preço de fechamento
* preço ajustado, 
* máximo diária,
* mínimo diário e
* volume.

In [26]:
# Vetor contendo os tickers dos ativos a serem analisados
symbols <- c("SPY", "EFA", "IJS", "EEM", "AGG")
# Downloa dos dados
prices <- getSymbols(symbols,
                     src="yahoo",
                     from="2012-12-31",
                     to="2017-12-31",
                     #auto.assign = TRUE,
                     warnings = FALSE) %>%
  map(~Ad(get(.))) %>%
  reduce(merge) %>%
  `colnames<-`(symbols)

Após baixar os dados de cada ativo, aplicamos `Ad(get(.))` para nos restringirmos apenas ao preço ajustado de cada ativo. As colunas resultantes são unidas por meio do comando `reduce(merge)`. Na célula seguinte, são mostradas as 3 primeiras linhas do resultado obtido.

In [27]:
head(prices,3)

                SPY      EFA      IJS      EEM      AGG
2012-12-31 120.1947 43.71408 35.37211 36.64785 89.67650
2013-01-02 123.2754 44.39063 36.34266 37.36677 89.57156
2013-01-03 122.9968 43.96011 36.29456 37.10234 89.34555

## Convertendo de preços diários para Retornos Mensais no paradigma `xts`

A função `to.montly` do pacote `quantmod`  converte os preços diários para preços mensais. Note-se o parâmetro `indexAt="lastof"` que toma os preços no último dia de cada mês (caso fosse utilizado o valor `"firstof"` para esse parâmetro, os preços seriam tomados no primeiro dia do mês). 

In [30]:
prices_monthly <- to.monthly(prices,
                             indexAt = 'lastof',
                             OHLC = FALSE)

In [18]:
head(prices_monthly,3)

                SPY      EFA      IJS      EEM      AGG
2012-12-31 120.1948 43.71409 35.37212 36.64785 89.67657
2013-01-31 126.3476 45.34393 37.26510 36.54043 89.11945
2013-02-28 127.9596 44.75966 37.87278 35.70583 89.64603

Finalmente, a função `Return.calculate` calcula o retorno do ativo. Como o parâmetro `method` foi utilizado com valor "log", temos o retorno logarítmo ($t_t$):

$$r_t = \log{\left(\dfrac{P_{t+1}}{P_t}\right)} = \log{(P_{t+1})} - \log{(P_t)}$$
em que $P_t$ é o valor do ativo no tempo $t$.

In [19]:
asset_returns_xts <- Return.calculate(prices_monthly,
                                     method = 'log') %>%
na.omit()

In [20]:
head(asset_returns_xts,3)

                  SPY         EFA        IJS          EEM           AGG
2013-01-31 0.04992310  0.03660596 0.05213315 -0.002935499 -0.0062319483
2013-02-28 0.01267764 -0.01296912 0.01617534 -0.023105309  0.0058913220
2013-03-31 0.03726845  0.01296912 0.04025793 -0.010234706  0.0009853806

## Convertendo preços diários para Retornos Mensais sob o paradigma `tityverse`

In [35]:
asset_returns_dplyr_byhand <- prices %>% 
    to.monthly(indexAt = "lastof", OHLC = FALSE) %>%
    # converte o index para uma data
    data.frame(date = index(.)) %>%
    # remove o index por que ele foi convertido para nome de linhas
    remove_rownames() %>%
    gather(asset, prices, -date) %>%
    group_by(asset) %>%
    mutate(returns = (log(prices) - log(lag(prices)))) %>%
    select(-prices) %>%
    spread(asset, returns) %>%
    select(date, symbols) %>%
    na.omit()

In [36]:
head(asset_returns_dplyr_byhand, 3)

date,SPY,EFA,IJS,EEM,AGG
2013-01-31,0.04992294,0.03660620,0.05213401,-0.002935499,-0.0062309101
2013-02-28,0.01267859,-0.01296921,0.01617464,-0.023104973,0.0058914883
2013-03-31,0.03726792,0.01296921,0.04025853,-0.010235240,0.0009845222


## Convertendo preços diários para Retornos Mensais sob o paradigma `tidyquant`

In [37]:
asset_returns_tq_builtin <- prices %>%
    tk_tbl(preserve_index=TRUE,
           rename_index="date") %>%
    gather(asset, prices, -date) %>%
    group_by(asset) %>%
    tq_transmute(mutate_fun=periodReturn,
                 period="monthly",
                 type="log") %>%
    spread(asset, montly.returns) %>%
    select(date, symbols) %>%
    slice(-1)

ERROR: Error in tk_tbl(., preserve_index = TRUE, rename_index = "date"): could not find function "tk_tbl"
